In [13]:
from pathlib import Path
import json
import polars as pl

data_dir = Path("../data/cruxeval_results_base_revisions/cruxeval_results_allenai/13B")

first_stage2 = [d for d in data_dir.iterdir() if "stage2" in d.name][0]

# Load the results
results = json.load(open(first_stage2))
results[0]


{'problem_id': 0,
 'is_correct': False,
 'full_generated_text': 'Given a function and an input, provide ONLY the output value that would result from executing the function with the given input.\nfunction: def f(nums):\n    output = []\n    for n in nums:\n        output.append((nums.count(n), n))\n    output.sort(reverse=True)\n    return output\ninput: [1, 1, 3, 1, 3, 1]\noutput:  [(2, 1), (2, 3)]',
 'generated': '[(2, 1), (2, 3)]',
 'true_output': '[(4, 1), (4, 1), (4, 1), (4, 1), (2, 3), (2, 3)]'}

In [14]:
df = pl.DataFrame([
    {
        "problem_id": result["problem_id"],
        "is_correct": result["is_correct"],
        "full_generated_text": result["full_generated_text"],
        "generated_tokens": result["generated"],
        "true_output": result["true_output"],
    }
    for result in results
])

df.head()

problem_id,is_correct,full_generated_text,generated_tokens,true_output
i64,bool,str,str,str
0,false,"""Given a function and an input,…","""[(2, 1), (2, 3)]""","""[(4, 1), (4, 1), (4, 1), (4, 1…"
1,false,"""Given a function and an input,…","""1""","""{1: None, 2: None}"""
2,false,"""Given a function and an input,…","""hbtofdeiequ""","""'hbtofdeiequ'"""
3,false,"""Given a function and an input,…","""bcksrutq""","""'bcksrutq'"""
4,false,"""Given a function and an input,…","""2""","""' '"""


In [16]:
correct_samples = df.filter(pl.col("is_correct")).sample(n=6)
incorrect_samples = df.filter(~pl.col("is_correct")).sample(n=6)

print("=== INTERLEAVED CORRECT AND INCORRECT PROBLEMS ===")
for i in range(6):  # Assuming both samples have 6 rows
    # Print a correct problem
    if i < len(correct_samples):
        correct_row = correct_samples.row(i)
        print(f"\nProblem {i + 1}A (CORRECT):")
        print("Full text:", correct_row[2])
        print("Generated:", correct_row[3])
        print("True output:", correct_row[4])
        print("-" * 80)

    # Print an incorrect problem
    if i < len(incorrect_samples):
        incorrect_row = incorrect_samples.row(i)
        print(f"\nProblem {i + 1}B (INCORRECT):")
        print("Full text:", incorrect_row[2])
        print("Generated:", incorrect_row[3])
        print("True output:", incorrect_row[4])
        print("-" * 80)

=== INTERLEAVED CORRECT AND INCORRECT PROBLEMS ===

Problem 1A (CORRECT):
Full text: Given a function and an input, provide ONLY the output value that would result from executing the function with the given input.
function: def f(a, b):
    for key, value in b.items():
        if key not in a:
            a[key] = [value]
        else:
            a[key].append(value)
    return a
input: {}, {'foo': 'bar'}
output:  {'foo': ['bar']}
Generated: {'foo': ['bar']}
True output: {'foo': ['bar']}
--------------------------------------------------------------------------------

Problem 1B (INCORRECT):
Full text: Given a function and an input, provide ONLY the output value that would result from executing the function with the given input.
function: def f(array, elem):
    result = array.copy()
    while result:
        key, value = result.popitem()
        if elem == key or elem == value:
            result.update(array)
        del result[key]
    return result
input: {}, 1
output: 1
Generated